Need to build the Transversional CNOT and CNOT gate

This let's use create logical CNOT gate 

Implement a logical CNOT as applying a CNOT on each corresponding physical qubit pair after encoding. 

Encode block A (control) and Encode block B (target)

CNOTc→t​=(I⊗H)CZ(I⊗H). H gate on target before, CZ control to target, H gate after on target



Process flow : 

- Encoding circuit (logical qubit) 
    - 7 qubit color/Steane state injection [[7,1,3]] 

- QEC (Steane) to create memory 

Encode Circuit (logical qubit)

![Encoding Circuit](Encode_circ_(logical_qubit).png)


Distillation circuit 

![Distillation Circuit](distillation_d3.png)

Used later (if at all)

In [1]:
# CNOT Gate between two logical qubits encoded with the Steane code
# 
# 
# import math

# def H(q): # H gate up to global phase
#     squin.u3(math.pi/2, 0.0, math.pi, q)

# def logical_cnot_blockwise(qc, qt):   # qc, qt are length-7 lists of qubits (encoded blocks)
#     for i in range(7):
#         H(qt[i])
#         squin.cz(qc[i], qt[i])
#         H(qt[i])

In [2]:
### Distillation Code ###

#call Logical qubit (encoded / error corrected) 

#squin.cz(q[1], q[2])
        # squin.sqrt_x(q[0])
        # squin.sqrt_x(q[1])
        # squin.sqrt_x(q[4])
        # squin.cz(q[0],q[1])
        # squin.cz(q[2],q[3])
        # squin.sqrt_y(q[0])
        # squin.sqrt_y(q[3])
        # squin.cz(q[0],q[2])
        # squin.cz(q[3],q[4])
        # squin.sqrt_x_adj(q[0]) 
        # squin.cz(q[1],q[3])
        # squin.cz(q[0],q[4])
        # for i in range(5):
        #    squin.sqrt_x_adj(q[i])

In [7]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [4]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [8]:
MSD_inq = MSD_encoding(0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_inq)
tsim_circ.diagram(height=400)

In [5]:
def error_detection(theta, phi):
    @squin.kernel 
    def parameterized_error_detection():
        MSD_enc = MSD_encoding(theta, phi)
        z_ancilla = MSD_encoding((np.pi)/2, 0) # Injected state is |+>
        z_measurement = []

        for i in range(7):
            squin.cx(MSD_enc.q[i], z_ancilla.q[i])
            z_measurement.append(squin.measure(z_ancilla.q[i]))
            # print(z_measurement[i])
        
        x_ancilla = MSD_encoding(0, 0) # Injected state is |0>
        x_measurement = []
        for i in range(7):
            squin.cx(x_ancilla.q[i], MSD_enc.q[i])
            squin.h(x_ancilla.q[i])
            squin.measure(x_ancilla.q[i])
            x_measurement.append(squin.measure(x_ancilla.q[i]))
            # print(x_measurement[i])
    return parameterized_error_detection

In [6]:
def MSD_encoding(theta, phi,basis="z"):
    @squin.kernel
    def z_MSD_encoding(q_subset): # MSD encoding of Z-basis ancilla (|+>); equivalent to the line z_ancilla = MSD_encoding((np.pi)/2, 0) before
        squin.u3((np.pi)/2,0,0,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q_subset[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q_subset[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
    @squin.kernel
    def parameterized_MSD_encoding(): # MSD encoding of both injected state and Z-basis ancilla
        q = squin.qalloc(14)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
        z_MSD_encoding([q[7], q[8], q[9], q[10], q[11], q[12], q[13]]) # Adding the Z-basis ancilla encoding
    
    return parameterized_MSD_encoding

In [ ]:
MSD_enc = MSD_encoding(0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)

In [8]:
def MSD_encoding_plus_ancilla(theta, phi,lam,basis="z"):
    @squin.kernel
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        q=squin.qalloc(14)
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi) # Z-basis ancilla encoding

        squin.cx(q[0],q[7])
        squin.cx(q[1],q[8])
        squin.cx(q[2],q[9])
        squin.cx(q[3],q[10])
        squin.cx(q[4],q[11])
        squin.cx(q[5],q[12])
        squin.cx(q[6],q[13])

        z_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        # z_measurement = []
        # z_measurement.append(squin.measure(q[7]))
        # z_measurement.append(squin.measure(q[8]))
        # z_measurement.append(squin.measure(q[9]))
        # z_measurement.append(squin.measure(q[10]))
        # z_measurement.append(squin.measure(q[11]))
        # z_measurement.append(squin.measure(q[12]))
        # z_measurement.append(squin.measure(q[13]))

        squin.broadcast.reset([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=0,phi=0,lam=0) # X-basis ancilla encoding

        squin.cx(q[7],q[0])
        squin.cx(q[8],q[1])
        squin.cx(q[9],q[2])
        squin.cx(q[10],q[3])
        squin.cx(q[11],q[4])
        squin.cx(q[12],q[5])
        squin.cx(q[13],q[6])

        squin.broadcast.h([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        x_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        # x_measurement = []
        # x_measurement.append(squin.measure(q[7]))
        # x_measurement.append(squin.measure(q[8]))
        # x_measurement.append(squin.measure(q[9]))
        # x_measurement.append(squin.measure(q[10]))
        # x_measurement.append(squin.measure(q[11]))
        # x_measurement.append(squin.measure(q[12]))
        # x_measurement.append(squin.measure(q[13]))

    return circ

In [9]:
MSD_enc = MSD_encoding_plus_ancilla(1,0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)

In [10]:
#CZ_L = CZ
#SQRT(Y)_L = SQRT(Y)
#SQRT(X)_L = SQRT(X)adj 



# --- 1) Encode helper ---

def big_encode(theta, phi,lam,basis="z"):
    @squin.kernel 
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):  # qb is length-7 list/slice: qb[0]..qb[6]
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])

    @squin.kernel
    def log_sqrt_x(q_subset):
        squin.sqrt_x_adj(q_subset[0]) 
        squin.sqrt_x_adj(q_subset[1])
        squin.sqrt_x_adj(q_subset[2])
        squin.sqrt_x_adj(q_subset[3])
        squin.sqrt_x_adj(q_subset[4])
        squin.sqrt_x_adj(q_subset[5])
        squin.sqrt_x_adj(q_subset[6])

    @squin.kernel
    def log_sqrt_x_adj(q_subset):
        squin.sqrt_x(q_subset[0]) 
        squin.sqrt_x(q_subset[1])
        squin.sqrt_x(q_subset[2])
        squin.sqrt_x(q_subset[3])
        squin.sqrt_x(q_subset[4])
        squin.sqrt_x(q_subset[5])
        squin.sqrt_x(q_subset[6])
    
    @squin.kernel
    def log_sqrt_y(q_subset):
        squin.sqrt_y_adj(q_subset[0]) 
        squin.sqrt_y_adj(q_subset[1])
        squin.sqrt_y_adj(q_subset[2])
        squin.sqrt_y_adj(q_subset[3])
        squin.sqrt_y_adj(q_subset[4])
        squin.sqrt_y_adj(q_subset[5])
        squin.sqrt_y_adj(q_subset[6])
    
    @squin.kernel
    def log_cz(q_subset):
        squin.cz(q_subset[0],q_subset[7])
        squin.cz(q_subset[1],q_subset[8])
        squin.cz(q_subset[2],q_subset[9])
        squin.cz(q_subset[3],q_subset[10])
        squin.cz(q_subset[4],q_subset[11])
        squin.cz(q_subset[5],q_subset[12])
        squin.cz(q_subset[6],q_subset[13])
    
    @squin.kernel
    def circ():
        q=squin.qalloc(35)
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=theta,phi=phi,lam=lam) # Z-basis ancilla encoding
        parameterized_MSD_encoding([q[14],q[15],q[16],q[17],q[18],q[19],q[20]],theta=theta,phi=phi,lam=lam) # X-basis ancilla encoding
        parameterized_MSD_encoding([q[21],q[22],q[23],q[24],q[25],q[26],q[27]],theta=theta,phi=phi,lam=lam) # Second Z-basis ancilla encoding
        parameterized_MSD_encoding([q[28],q[29],q[30],q[31],q[32],q[33],q[34]],theta=theta,phi=phi,lam=lam) # Second X-basis ancilla encoding
        log_sqrt_x([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        log_sqrt_x([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])
        log_sqrt_x([q[28],q[29],q[30],q[31],q[32],q[33],q[34]])
        log_cz([q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13]])
        log_cz([q[14],q[15],q[16],q[17],q[18],q[19],q[20],q[21],q[22],q[23],q[24],q[25],q[26],q[27]])
        log_sqrt_y([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        log_sqrt_y([q[21],q[22],q[23],q[24],q[25],q[26],q[27]])
        log_cz([q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[14],q[15],q[16],q[17],q[18],q[19],q[20]])
        log_cz([q[21],q[22],q[23],q[24],q[25],q[26],q[27],q[28],q[29],q[30],q[31],q[32],q[33],q[34]])
        log_sqrt_x_adj([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        log_cz([q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[21],q[22],q[23],q[24],q[25],q[26],q[27]])
        log_cz([q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[28],q[29],q[30],q[31],q[32],q[33],q[34]])
        log_sqrt_x_adj([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        log_sqrt_x_adj([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])
        log_sqrt_x_adj([q[14],q[15],q[16],q[17],q[18],q[19],q[20]])
        log_sqrt_x_adj([q[21],q[22],q[23],q[24],q[25],q[26],q[27]])
        log_sqrt_x_adj([q[28],q[29],q[30],q[31],q[32],q[33],q[34]])
        if basis=="z":
            squin.broadcast.measure([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        elif basis=="x":
            squin.broadcast.h([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
            squin.broadcast.measure([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        elif basis=="y":
            squin.broadcast.sqrt_x([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
            squin.broadcast.h([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
            squin.broadcast.measure([q[0],q[1],q[2],q[3],q[4],q[5],q[6]])
        squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15],q[16],q[17],q[18],q[19],q[20],q[21],q[22],q[23],q[24],q[25],q[26],q[27],q[28],q[29],q[30],q[31],q[32],q[33],q[34]])
    return circ





In [11]:
big_enc = big_encode(np.pi/2,0,0,basis="z")
tsim_circ = bloqade.tsim.Circuit(big_enc)
tsim_circ.diagram(height=400)

In [45]:
theta = np.pi/2
phi = np.pi/4
lam = 0
big_enc = big_encode(theta,phi,lam,basis="z")
tsim_circ = bloqade.tsim.Circuit(big_enc)
sampler = tsim_circ.compile_sampler()
samples = sampler.sample(shots=1000)
result = 1 - 2 * samples.astype(int)
Zexpval = np.mean(np.array([i[0]*i[1]*i[5] for i in result if i[7]*i[8]*i[12]==-1 and i[14]*i[15]*i[19]==1 and i[21]*i[22]*i[26]==-1 and i[28]*i[29]*i[33]==-1]))
print(f"ZExpVal:{Zexpval}")
big_enc = big_encode(theta,phi,lam,basis="x")
tsim_circ = bloqade.tsim.Circuit(big_enc)
sampler = tsim_circ.compile_sampler()
samples = sampler.sample(shots=1000)
result = 1 - 2 * samples.astype(int)
Xexpval = np.mean(np.array([-i[0]*i[1]*i[5] for i in result if i[7]*i[8]*i[12]==-1 and i[14]*i[15]*i[19]==1 and i[21]*i[22]*i[26]==-1 and i[28]*i[29]*i[33]==-1]))
print(f"XExpVal:{Xexpval}")
big_enc = big_encode(theta,phi,lam,basis="y")
tsim_circ = bloqade.tsim.Circuit(big_enc)
sampler = tsim_circ.compile_sampler()
samples = sampler.sample(shots=1000)
result = 1 - 2 * samples.astype(int)
YExpVal = np.mean(np.array([-i[0]*i[1]*i[5] for i in result if i[7]*i[8]*i[12]==-1 and i[14]*i[15]*i[19]==1 and i[21]*i[22]*i[26]==-1 and i[28]*i[29]*i[33]==-1]))
print(f"YExpVal:{YExpVal}")

ZExpVal:-0.09090909090909091
XExpVal:0.7777777777777778
YExpVal:0.7872340425531915


In [46]:
truex=np.sin(theta)*np.cos(phi)
truey=np.sin(theta)*np.sin(phi)
truez=np.cos(theta)
print(f"TrueX:{truex}, TrueY:{truey}, TrueZ:{truez}")

TrueX:0.7071067811865476, TrueY:0.7071067811865475, TrueZ:6.123233995736766e-17


In [47]:
F=0.5*(1 + Xexpval*truex + YExpVal*truey + Zexpval*truez)

In [48]:
print(F)

1.0533152353965656
